In [1]:
import pandas as pd
import os
os.chdir("..")
kpis = pd.read_csv("simulation_data/gateway_viability_sweep_ag1_.csv", index_col=0)
from adaptive_multigrid import give_data_individual_names, find_best_individual_name
from gvs_threshold_calc import calc_gvs_thresh_ineq_met

/Users/seanmcowen/opt/anaconda3/envs/BlockScience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fitness_func = calc_gvs_thresh_ineq_met

In [3]:
"""old_param_grid = deepcopy(new_param_grid)
old_best_name = new_best_name
old_best_fitness = new_best_fitness

individual_dict = build_individual_dict_from_param_grid(old_param_grid)
print(individual_dict)

# TODO: update from mock to actual."""
data = kpis
data = give_data_individual_names(data)

new_best_name, new_best_val = find_best_individual_name(
            data, fitness_func=fitness_func, name_biased_towards=old_best_name
        )

assert not (
            new_best_name is None
        ), "The best individual should be somewhere in this dictionary."
print(new_best_name)

new_best_individual = individual_dict[new_best_name]

if new_best_name == old_best_name:
    times_repeat_best_name = times_repeat_best_name + 1
else:
    times_repeat_best_name = 0

if isclose(new_best_fitness, old_best_fitness):
    times_repeat_best_fitness = times_repeat_best_fitness + 1
else:
    times_repeat_best_fitness = 0

Finding names to use for individual.
We have added individual names for the various parameter combinations.
The DataFrame now looks like                                                     Net Minting Rate  \
individual_name                                                        
afpr:20_gfpr:20_gms:100000000000.0_session_toke...          0.001651   
afpr:20_gfpr:20_gms:100000000000.0_session_toke...          0.001692   
afpr:20_gfpr:20_gms:100000000000.0_session_toke...          0.001698   
afpr:20_gfpr:20_gms:100000000000.0_session_toke...          0.001704   
afpr:20_gfpr:20_gms:100000000000.0_session_toke...          0.005313   

                                                       KPI C  \
individual_name                                                
afpr:20_gfpr:20_gms:100000000000.0_session_toke...  0.159556   
afpr:20_gfpr:20_gms:100000000000.0_session_toke...  0.182310   
afpr:20_gfpr:20_gms:100000000000.0_session_toke...  0.164375   
afpr:20_gfpr:20_gms:100000000000.0_ses

NameError: name 'old_best_name' is not defined